In [1]:
%matplotlib inline

%reload_ext autoreload
%autoreload 2

In [2]:
# Specify visible cuda device
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from parameters import *
from lib.utils import *
from lib.attacks import *

import numpy as np
import tensorflow as tf

/home/chawins/.conda/envs/tsa3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/chawins/.conda/envs/tsa3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [42]:
def fn_mnist_cnn_hinge(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[3, 3],
        activation=tf.nn.relu)

    # Convolutional Layer and pooling #2
    conv2 = tf.layers.conv2d(
        inputs=conv1,
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dropout #1
    drop1 = tf.layers.dropout(
        inputs=pool1, rate=0.25, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    # Dense Layer
    drop1_flat = tf.reshape(drop1, [-1, 12 * 12 * 64])
    dense = tf.layers.dense(inputs=drop1_flat, units=128, activation=tf.nn.relu)
    drop2 = tf.layers.dropout(
        inputs=dense, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))

    # Logits Layer
    logits = tf.layers.dense(inputs=drop2, units=10, name="logits")

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
 
    # Structured hinge loss max{0, 1 - (y_label - y_max)}
    # Not so elegant way to index tensor with another tensor
    indices = tf.range(tf.shape(logits)[0])
    gather_ind = tf.stack([indices, labels], axis=1)
    y_label = tf.gather_nd(logits, gather_ind)
    # Get 2 largest outputs
    y_2max = tf.nn.top_k(logits, 2)[0]
    # Find y_max = max(z[i != y])
    i_max = tf.to_int32(tf.argmax(logits, axis=1))
    y_max = tf.where(tf.equal(labels, i_max), y_2max[:, 1],
                     y_2max[:, 0])
    loss = tf.reduce_sum(tf.maximum(0., 1. - y_label + y_max))
    
    # Calculate batch accuracy
    tmp = tf.cast(tf.equal(i_max, labels), dtype=tf.float32)
    accuracy = tf.reduce_mean(tmp, name="accuracy")
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [43]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train[:, :, :, np.newaxis]
x_test = x_test[:, :, :, np.newaxis]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
#y_train = keras.utils.to_categorical(y_train, NUM_LABELS)
#y_test = keras.utils.to_categorical(y_test, NUM_LABELS)

In [44]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=fn_mnist_cnn_hinge, model_dir="./tmp/mnist_cnn_hinge")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/mnist_cnn_hinge', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd879da3a58>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
#tensors_to_log = {"probabilities": "softmax_tensor"}
#tensors_to_log = {"gather_ind": "gather_ind", "y_l": "y_l"}
tensors_to_log = {"accuracy": "accuracy"}
#tensors_to_log = {}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)

In [46]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=128,
    num_epochs=12,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./tmp/mnist_cnn_hinge/model.ckpt.
INFO:tensorflow:accuracy = 0.0546875
INFO:tensorflow:loss = 152.7504, step = 1
INFO:tensorflow:global_step/sec: 117.879
INFO:tensorflow:accuracy = 0.9296875 (0.850 sec)
INFO:tensorflow:loss = 19.594309, step = 101 (0.849 sec)
INFO:tensorflow:global_step/sec: 131.897
INFO:tensorflow:accuracy = 0.9453125 (0.758 sec)
INFO:tensorflow:loss = 20.482437, step = 201 (0.758 sec)
INFO:tensorflow:global_step/sec: 132.394
INFO:tensorflow:accuracy = 0.9765625 (0.755 sec)
INFO:tensorflow:loss = 12.634214, step = 301 (0.756 sec)
INFO:tensorflow:global_step/sec: 125.713
INFO:tensorflow:accuracy = 0.9609375 (0.796 sec)
INFO:tensorflow:loss = 14.694176, step = 401 (0.796 sec)
INFO:tensorflow:global_step/sec: 127.816
INFO:tensorflow:accuracy = 0.9375 (0.782 sec)
INFO:tensorflow:loss = 19.486855, step = 501 (0.782 sec)
INFO:tensorflow:global_step/sec: 126.336
INFO:tensorflow:accuracy

INFO:tensorflow:global_step/sec: 129.416
INFO:tensorflow:accuracy = 0.9921875 (0.772 sec)
INFO:tensorflow:loss = 1.4496298, step = 5601 (0.772 sec)
INFO:tensorflow:Saving checkpoints for 5625 into ./tmp/mnist_cnn_hinge/model.ckpt.
INFO:tensorflow:Loss for final step: 2.9344773.


In [47]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-03-24-22:56:53
INFO:tensorflow:Restoring parameters from ./tmp/mnist_cnn_hinge/model.ckpt-5625
INFO:tensorflow:Finished evaluation at 2018-03-24-22:56:53
INFO:tensorflow:Saving dict for global step 5625: accuracy = 0.9903, global_step = 5625, loss = 3.4338567
{'accuracy': 0.9903, 'loss': 3.4338567, 'global_step': 5625}


In [54]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [55]:
def get_weights(estimator):
    """
    Extract weights from TF Estimator. Only works with a simple CNN/DNN.
    """

    weights = []
    weight = []
    layer_names = estimator.get_variable_names()
    for layer_name in layer_names:
        if layer_name.endswith("kernel"):
            weight.insert(0, estimator.get_variable_value(layer_name))
            weights.append(weight)
            weight = []
        elif layer_name.endswith("bias"):
            weight.append(estimator.get_variable_value(layer_name))
            
    return weights


def load_weights(model, weights):
    """
    Set weights in Keras model with a list of weights.
    """
    
    i = 0
    for layer in model.layers:
        # Check if layer has trainable weights
        if not layer.trainable_weights:
            continue
        # Set weight
        layer.set_weights(weights[i])
        i += 1
        
    assert i == len(weights), "Number of layers mismatch."

In [56]:
weights = get_weights(mnist_classifier)
load_weights(model, weights)
model.evaluate(x_train, y_train)

In [61]:
model.save('./tmp/mnist_cnn_hinge_keras.h5')

In [62]:
model = keras.models.load_model('./tmp/mnist_cnn_hinge_keras.h5')

In [ ]:
# Can we change margin? it doesn't have to be 1